In [788]:
import pandas as pd
from collections import Counter
import numpy as np
import math
from functools import partial
from sklearn.model_selection import train_test_split

In [789]:
df = pd.read_csv("titanic-homework.csv").drop(['PassengerId', 'Name'], axis=1)
df

,Pclass,Sex,Age,SibSp,Parch,Survived
0,3,male,22,1,0,0
1,1,female,38,1,0,1
2,3,female,26,0,0,1
3,1,female,35,1,0,1
4,3,male,35,0,0,0
...,...,...,...,...,...,...
95,3,male,44,0,0,0
96,1,male,71,0,0,0
97,1,male,23,0,1,1
98,2,female,34,0,1,1


In [790]:
# podział na podzbiory treningowy i testowy
train, test = train_test_split(df, test_size=0.2)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [791]:
def entropy(s):
    labels = Counter([e[1] for e in s])
    p = np.array(list(labels.values())) / labels.total()
    return -sum([p_i * math.log2(p_i) if p_i != 0 else 0 for p_i in p])

In [792]:
def cond_entropy(Z):
    sizes = [len(z) for z in Z]
    total = sum(sizes)
    freqs = np.array(sizes) / total
    entropies = np.array([entropy(z) for z in Z])
    return freqs.dot(entropies)

In [793]:
def gain(initial, cond):
    return initial - cond

In [794]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [795]:
class NumericDivision():
    def __init__(self, boundary=0, smaller=None, greater_or_equal=None):
        self.boundary = boundary
        self.smaller = [] if smaller is None else smaller
        self.greater_or_equal = [] if greater_or_equal is None else greater_or_equal

    def items(self):
        return [self.smaller, self.greater_or_equal]

    def __str__(self):
        return f'NumericDivision(\n\tboundary={self.boundary},\n\tsmaller={self.smaller},\n\tgreater_or_equal={self.greater_or_equal}\n)'

In [796]:
class ClassDivision():
    def __init__(self, edges=None):
        self.edges = {} if edges is None else edges

    def items(self):
        return [v for k, v in self.edges.items()]

    def __str__(self):
        return f'ClassDivision({self.edges})'

In [797]:
# sprawdzenie czy podzbiór [(id, dezycja)] jest  liściem
# jest liściem jak wszystkie decyzje są takie same albo
# jak jest pusty
def is_leaf(subset):
    if len(subset) == 0:
        return True
    last = subset[0][1]
    for i in range(1, len(subset)):
        if subset[i][1] != last:
            return False
    return True

In [798]:
# zwrócenie wartości liścia
# jak podzbiór jest pusty to zwracamy najpopularniejszą wartość decyzyjną
def leaf_value(subset, data, decision):
    if len(subset) == 0:
        return Counter(data[decision].tolist()).most_common()[0][0]
    return subset[0][1]

In [799]:
# szukanie kandydatów do podziału podzbioru według kolumny liczbowej
def best_numeric_division_candidates(S, data, feature, decision):
    ids = [s[0] for s in S]
    subdata = data[data.index.isin(ids)].sort_values(feature)
    candidates = []
    last_label = subdata.iloc[0][decision]
    for i, row in subdata.iterrows():
        if last_label != row[decision]:
            candidates.append(row[feature])
    return sorted(list(set(candidates)))

In [800]:
# użycie kandydatów i sprawdzenie, który najlepiej dzieli podzbiór
def find_best_numeric_division(candidates, S, data, feature, decision):
    ids = [s[0] for s in S]
    subdata = data[data.index.isin(ids)]
    best_division = None
    best_entropy = float('inf')
    for candidate in candidates:
        division = NumericDivision(candidate)
        for i, row in subdata.iterrows():
            if row[feature] >= candidate:
                division.greater_or_equal.append((i, row[decision]))
            else:
                division.smaller.append((i, row[decision]))
        ce = cond_entropy(division.items())
        if ce < best_entropy:
            best_entropy = ce
            best_division = division
    return best_division

In [801]:
def divide(S, data, feature, decision):
    ids = [s[0] for s in S]
    if data.dtypes[feature] in numerics:
        candidates = best_numeric_division_candidates(S, data, feature, decision)
        return find_best_numeric_division(candidates, S, data, feature, decision)
    else:
        division = ClassDivision()
        for i, row in data[data.index.isin(ids)].iterrows():
            value = row[feature]
            if not value in division.edges:
                division.edges[value] = []
            division.edges[value].append((i, row[decision]))
        return division

In [802]:
class ClassNode():
    def __init__(self, name):
        self.name = name
        self.edges = {}

    def add_edge(self, value, node):
        self.edges[value] = node

    def infere(self, row):
        return self.edges[row[self.name]].infere(row)

    def str_with_intends(self, intends):
        return f'''ClassNode(
{' ' * intends}    name: {self.name},
{' ' * intends}    edges: {{
{' ' * intends}        {',\n        '.join([x[0]+": "+x[1].str_with_intends(intends+8) for x in self.edges.items()])}
{' ' * intends}    }}
{' ' * intends})'''

    def __str__(self):
        return self.str_with_intends(0)

In [803]:
class IntervalNode():
    def __init__(self, name, boundary):
        self.name = name
        self.boundary = boundary
        self.smaller = None
        self.greater_or_equal = None

    def set_smaller_node(self, node):
        self.smaller = node

    def set_greater_or_equal_node(self, node):
        self.greater_or_equal = node

    def infere(self, row):
        return self.smaller.infere(row) \
                if row[self.name] < self.boundary \
                else self.greater_or_equal.infere(row)

    def str_with_intends(self, intends):
        return f'''IntervalNode(
{' ' * intends}    name: {self.name},
{' ' * intends}    tboundary: {self.boundary},
{' ' * intends}    smaller: {self.smaller.str_with_intends(intends+4)},
{' ' * intends}    greater_or_equal: {self.greater_or_equal.str_with_intends(intends+4)}
{' ' * intends})'''

    def __str__(self):
        return self.str_with_intends(0)

In [804]:
class LeafNode():
    def __init__(self, value):
        self.value = value

    def infere(self, row):
        return self.value

    def str_with_intends(self, intends):
        return str(self)
    
    def __str__(self):
        return f'LeafNode({self.value})'

In [805]:
# funkcja budująca drzewo
# może pracować w nieskończoność jak wszystkie podziały dadzą w dwóch turach taki sam zysk :(
def build_tree(data, decision):
    model = None

    # funkcja ustawiania zmiennej modelu powyżej
    def set_model(node):
        nonlocal model
        model = node

    # sprawdź czy podzbiór z podziału jest liściem
    # jak tak to go ustaw w drzewie przy pomocy metody ustawiania
    # jak nie to dodaj ten podzbiór na kolejkę do dalszego przetworzenia
    def check_node(division_items, column, set_node, cur_columns):
        if is_leaf(division_items) or len(cur_columns) == 1:
            set_node(LeafNode(leaf_value(division_items, data, decision)))
        else:
            nodes.append((division_items, column, set_node, cur_columns))
    
    S = [(i, row[decision]) for (i, row) in data.iterrows()]


    # wywal kolumnę decyzyjną z kolumn,
    # żeby nie brać jej pod uwagę przy budowaniu drzewa
    columns = data.columns.drop(decision)

    # podzbiór, cecha, metoda ustawiania node w drzewie
    # zaczynamy z jednym elementem w kolejce, żeby nasz program działał
    # cały zbiór jako podzbiór,
    # pusta aktualna kolumna bo będziemy tworzyć korzeń drzewa
    # przekazujemy funkcję ustawiania modelu jako ostatni parametr
    # potem przekażemy takie funkcje, które ustawiają konkretne
    # krawędzie drzewa, wraz z tym jak będziemy go budować.
    nodes = [(S, None, set_model, columns)]

    # pracuj do póki kolejka nie będzie pusta
    while len(nodes) > 0:
        # pobranie pierwszego elementu z kolejki
        S, feature, set_node, cur_columns = nodes.pop(0)

        # policz entropię potrzebną do gaina
        initial_entropy = entropy(S)

        # wywal kolumnę z najbliższego korzenia
        # nie chcemy powtarzać w kółko tego samego podziału
        cur_columns = cur_columns.drop(feature) if feature is not None else columns

        # szukamy najlepszego podziału
        best_gain = float('-inf')
        best_division = None
        best_column = None
        for column in cur_columns:
            division = divide(S, data, column, decision)

            curr_gain = gain(initial_entropy, cond_entropy(division.items()))
            if curr_gain > best_gain:
                best_gain = curr_gain
                best_division = division
                best_column = column

        # jeśli podział jest podziałem liczbowym
        if type(best_division) is NumericDivision:

            node = IntervalNode(best_column, best_division.boundary)
            # sprawdź czy podzbiór podziału danej krawędzi jest liściem i jak tak to go dodaj
            # a jak nie to dodaj ten podzbiór na kolejkę do przetworzenia (to samo w kolejnej linii)
            check_node(best_division.smaller, best_column, node.set_smaller_node, cur_columns)
            check_node(best_division.greater_or_equal, best_column, node.set_greater_or_equal_node, cur_columns)

        else: # jeśli podział jest podziałem na klasy tekstowe
            node = ClassNode(best_column)
            # sprawdź czy podzbiór podziału danej krawędzi jest liściem i jak tak to go dodaj
            # a jak nie to dodaj ten podzbiór na kolejkę do przetworzenia
            for k, v in best_division.edges.items():
                check_node(best_division.edges[k], best_column, partial(ClassNode.add_edge, node, k), cur_columns)

            # zapewnij, że w drzewie będą krawędzie, których już brakuje w podzbiorze zbioru danych
            for k in (set(data[best_column].unique().tolist()) - set(best_division.edges.keys())):
                node.add_edge(k, LeafNode(leaf_value([], data, decision)))


        # ustaw krawędź pobranego node'a
        set_node(node)
        
    return model

In [806]:
model = build_tree(train, 'Survived')
print(model)

ClassNode(
    name: Sex,
    edges: {
        female: IntervalNode(
            name: SibSp,
            tboundary: 1,
            smaller: LeafNode(1),
            greater_or_equal: IntervalNode(
                name: Pclass,
                tboundary: 3,
                smaller: LeafNode(1),
                greater_or_equal: IntervalNode(
                    name: Age,
                    tboundary: 33,
                    smaller: LeafNode(0),
                    greater_or_equal: IntervalNode(
                        name: Parch,
                        tboundary: 0,
                        smaller: LeafNode(0),
                        greater_or_equal: LeafNode(0)
                    )
                )
            )
        ),
        male: IntervalNode(
            name: Pclass,
            tboundary: 3,
            smaller: IntervalNode(
                name: Age,
                tboundary: 19,
                smaller: LeafNode(1),
                greater_or_equal: IntervalNod

In [807]:
correct = 0
for _, row in test.iterrows():
    output = model.infere(row)
    if output == row['Survived']:
        correct += 1

print("Test accuracy:", correct / test.shape[0])

Test accuracy: 0.75


In [808]:
correct = 0
for _, row in train.iterrows():
    output = model.infere(row)
    if output == row['Survived']:
        correct += 1

print("Train accuracy:", correct / train.shape[0])

Train accuracy: 0.875


# Podsumowanie wyników

Jest dobrze. Dokładność klasyfikacji dla zbioru testowego jest powyżej 80%.

In [809]:
def enumerate_tree(node, depth=0):
    if isinstance(node, ClassNode):
        print('|     '*(depth),'|---- ', node.name, sep='')
        for edgeName, child in node.edges.items():
            print('|     '*(depth+1),'|---- ', edgeName, sep='')
            enumerate_tree(child, depth+1)
    elif isinstance(node, IntervalNode):
        print('|     '*(depth+1),'|---- ', node.name, ' < ', node.boundary, sep='')
        print('|     '*(depth+2),'|---- ', "YES", sep='')
        enumerate_tree(node.smaller, depth+2)
        print('|     '*(depth+2),'|---- ', "NO", sep='')
        enumerate_tree(node.greater_or_equal, depth+2)
    elif isinstance(node, LeafNode):
        print('|     '*(depth+1),'|---- ', node.value, sep='')

In [810]:
enumerate_tree(model)

|---- Sex
|     |---- female
|     |     |---- SibSp < 1
|     |     |     |---- YES
|     |     |     |     |---- 1
|     |     |     |---- NO
|     |     |     |     |---- Pclass < 3
|     |     |     |     |     |---- YES
|     |     |     |     |     |     |---- 1
|     |     |     |     |     |---- NO
|     |     |     |     |     |     |---- Age < 33
|     |     |     |     |     |     |     |---- YES
|     |     |     |     |     |     |     |     |---- 0
|     |     |     |     |     |     |     |---- NO
|     |     |     |     |     |     |     |     |---- Parch < 0
|     |     |     |     |     |     |     |     |     |---- YES
|     |     |     |     |     |     |     |     |     |     |---- 0
|     |     |     |     |     |     |     |     |     |---- NO
|     |     |     |     |     |     |     |     |     |     |---- 0
|     |---- male
|     |     |---- Pclass < 3
|     |     |     |---- YES
|     |     |     |     |---- Age < 19
|     |     |     |     |     |---- YES
| 